In [1]:
from transformers import ViTImageProcessor, ViTForImageClassification

model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name_or_path)

In [2]:
# train_path = "E:/casper/raw_data_training/test/"
# val_path = "E:/casper/raw_data_training/val/"
path = "E:/GLO_林口長庚資料集/Process_2022/Data/"

In [3]:
import os
# print(os.listdir(train_path + "MN"))

In [67]:
from datasets import load_dataset
dataset = load_dataset("imagefolder", data_dir=path)

Resolving data files:   0%|          | 0/11399 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1376 [00:00<?, ?it/s]

Found cached dataset imagefolder (C:/Users/VLSI/.cache/huggingface/datasets/imagefolder/default-8787100763c9607f/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/2 [00:00<?, ?it/s]

In [79]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 11399
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 1376
    })
})

In [209]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')
    # print(intputs)
    # print(intputs
    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    inputs['pixel_values'] = [ii.to(device) for ii in inputs['pixel_values']]
        
    # inputs['pixel_values'] = inputs['pixel_values'].to(device)
    # for ii in inputs['labels']:
    #     ii.to(device)
    # # print(inputs)
    return inputs

In [210]:
print(dataset)
prepared_ds = dataset.with_transform(transform)
print(prepared_ds)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 11399
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 1376
    })
})
DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 11399
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 1376
    })
})


In [211]:
from transformers import ViTForImageClassification

labels = ['IgA','MN']

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)
model.to("cuda")

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 11.99 GiB total capacity; 10.45 GiB already allocated; 0 bytes free; 11.16 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-ggWITHCUDA",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [ ]:
prepared_ds

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=processor,
)

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [ ]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
def Notification(subject,message):
    content = MIMEMultipart()  
    token = "dpyj vfsd fhhg fhfo"
    email = "casperliu376@gmail.com"
    content["subject"] = subject
    content["from"] = "VLSI machine"
    content["to"] = "casperliu1118@gmail.com"
    content.attach(MIMEText(message))
    with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:
        try:
            smtp.ehlo()
            smtp.starttls()
            smtp.login(email, token)
            smtp.send_message(content)
            print("Complete!")
        except Exception as e:
            print("Error message: ", e)

In [ ]:
mes = """HI CASPER,

    I AM TRANSFORMER, PLEASE CONTACT ME.

SINCERELY,
VLSI
"""
sub = "VIT"
Notification(sub,mes)